In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models, optimizers, utils, datasets

NUM_CLASSES = 10

/Users/priyarajni/Desktop/Pythoncode/MSAI-630-A02/venvMLP/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/Users/priyarajni/Desktop/Pythoncode/MSAI-630-A02/venvMLP/lib/python3.13/site-packages/keras/src/datasets/cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


In [4]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [5]:
y_train = utils.to_categorical(y_train, NUM_CLASSES)
y_test = utils.to_categorical(y_test, NUM_CLASSES)

In [6]:
CLASSES = np.array(
    ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
)

In [7]:
def build_mlp(activation_name: str):
    input_layer = layers.Input((32, 32, 3))
    x = layers.Flatten()(input_layer)

    if activation_name.lower() == "leakyrelu":
        x = layers.Dense(200)(x)
        x = layers.LeakyReLU(negative_slope=0.01)(x)
        x = layers.Dense(150)(x)
        x = layers.LeakyReLU(negative_slope=0.01)(x)
    else:
        # "relu" or "sigmoid"
        x = layers.Dense(200, activation=activation_name)(x)
        x = layers.Dense(150, activation=activation_name)(x)

    output_layer = layers.Dense(NUM_CLASSES, activation="softmax")(x)
    model = models.Model(input_layer, output_layer)

    opt = optimizers.Adam(learning_rate=0.0005)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model

activations = ["relu", "leakyrelu", "sigmoid"]
results = {}

for act in activations:
    print("\n==============================")
    print(f"Training model with: {act}")
    print("==============================")

    model = build_mlp(act)
    history = model.fit(
        x_train, y_train,
        batch_size=32,
        epochs=10,
        shuffle=True,
        validation_split=0.1,
        verbose=1
    )

    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    results[act] = {
        "history": history.history,
        "test_loss": test_loss,
        "test_acc": test_acc,
        "model": model
    }


Training model with: relu
Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2359 - loss: 2.0436 - val_accuracy: 0.3026 - val_loss: 1.9355
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3170 - loss: 1.8949 - val_accuracy: 0.3278 - val_loss: 1.8718
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3460 - loss: 1.8305 - val_accuracy: 0.3564 - val_loss: 1.8141
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3645 - loss: 1.7730 - val_accuracy: 0.3680 - val_loss: 1.7645
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3794 - loss: 1.7234 - val_accuracy: 0.3854 - val_loss: 1.7235
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3935 - loss: 1.6840 - val_accuracy: 0.3982 - val_loss: 1.6858
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4047 - loss: 1.6522 - val_accuracy: 0.4048 - val_loss: 1.6613
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accurac

In [8]:
print("\nFinal Test Results")
for act in activations:
    print(f"{act:10s}  loss={results[act]['test_loss']:.4f}  acc={results[act]['test_acc']:.4f}")


Final Test Results
relu        loss=1.5656  acc=0.4418
leakyrelu   loss=1.4707  acc=0.4778
sigmoid     loss=1.8711  acc=0.3271


In [9]:
best_act = max(activations, key=lambda a: results[a]["test_acc"])
best_model = results[best_act]["model"]

preds = best_model.predict(x_test, verbose=0)
preds_single = CLASSES[np.argmax(preds, axis=-1)]
actual_single = CLASSES[np.argmax(y_test, axis=-1)]

n_to_show = 10
indices = np.random.choice(range(len(x_test)), n_to_show, replace=False)

fig = plt.figure(figsize=(15, 3))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

<Figure size 1500x300 with 0 Axes>

In [10]:
for i, idx in enumerate(indices):
    img = x_test[idx]
    ax = fig.add_subplot(1, n_to_show, i + 1)
    ax.axis("off")
    ax.text(0.5, -0.35, "pred = " + str(preds_single[idx]),
            fontsize=10, ha="center", transform=ax.transAxes)
    ax.text(0.5, -0.7, "act = " + str(actual_single[idx]),
            fontsize=10, ha="center", transform=ax.transAxes)
    ax.imshow(img)

plt.show()
print(f"\nDisplayed predictions using best model: {best_act}")


Displayed predictions using best model: leakyrelu
